In [1]:
import qiskit

In [2]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()

qiskit_runtime_service.__init__:WARNING:2025-11-15 08:19:48,497: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: open-instance. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().


In [3]:
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2

# Load IBM account
service = QiskitRuntimeService()

# Select backend (hardware)
backend = service.backend("ibm_fez")

# Make circuit
qc = QuantumCircuit(1, 1)
qc.h(0)
qc.measure(0, 0)

# ⭐ Transpile for target hardware ⭐
qc_t = transpile(qc, backend)

# Run on hardware through SamplerV2
sampler = SamplerV2(backend)
job = sampler.run([qc_t])   # submit job

import time
print("Job submitted. Status:", job.status())
while not job.done():
    print("Job status:", job.status())
    time.sleep(10)  # Waits 10 seconds before next check

result = job.result()
print("Job finished!", job.status())
print(result)




qiskit_runtime_service.__init__:WARNING:2025-11-15 08:20:27,497: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: open-instance. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2025-11-15 08:20:27,501: Using instance: open-instance, plan: open


Job submitted. Status: QUEUED
Job status: RUNNING
Job finished! DONE
PrimitiveResult([SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=4096, num_bits=1>)), metadata={'circuit_metadata': {}})], metadata={'execution': {'execution_spans': ExecutionSpans([DoubleSliceSpan(<start='2025-11-15 02:50:34', stop='2025-11-15 02:50:38', size=4096>)])}, 'version': 2})


In [ ]:
# alice_qiskit.py
from flask import Flask, request, jsonify
import numpy as np, time, requests, random
# We'll try to use qiskit primitives; if not available we fall back to analytic simulator
try:
    from qiskit import QuantumCircuit, transpile
    from qiskit.primitives import Sampler
    _HAS_SAMPLER = True
except Exception:
    # sampler not available (common on Windows without Aer); we will use physics-faithful fallback
    from qiskit import QuantumCircuit  # only for clarity (we won't run circuits)
    _HAS_SAMPLER = False

app = Flask(__name__)

# change ALICE_IP if you run across machines; for single-machine use "127.0.0.1"
ALICE_IP = "192.168.1.10"
EVE_HOST = f"http://192.168.1.10:5001"
BOB_HOST = f"http://192.168.1.10:5002"

N = 64  # number of qubits per protocol run

alice_bits = None
alice_bases = None
final_key = None

@app.route("/start_protocol", methods=["POST"])
def start_protocol():
    global alice_bits, alice_bases
    np.random.seed(int(time.time()) % 100000)
    alice_bits = np.random.randint(2, size=N).tolist()
    alice_bases = np.random.randint(2, size=N).tolist()
    payload = [{"idx": i, "bit": int(alice_bits[i]), "basis": int(alice_bases[i])} for i in range(N)]
    print(f"Alice: sending {N} prepared qubits (classical descriptions) to Eve")
    try:
        r = requests.post(f"{EVE_HOST}/receive_from_alice", json={"preps": payload}, timeout=30)
        return jsonify({"status": "sent_to_eve", "eve_reply": r.text})
    except Exception as e:
        print("Alice -> Eve failed:", e)
        return jsonify({"error": str(e)}), 500

@app.route("/announce_bases", methods=["GET"])
def announce_bases():
    global alice_bases
    if alice_bases is None:
        return jsonify({"error": "no session active"}), 400
    print("Alice: announcing bases to Bob")
    try:
        r = requests.post(f"{BOB_HOST}/receive_alice_bases", json={"bases": alice_bases}, timeout=30)
        return jsonify({"status": "bases_sent", "bob_reply": r.text})
    except Exception as e:
        print("Alice -> Bob failed:", e)
        return jsonify({"error": str(e)}), 500

@app.route("/set_final_key", methods=["POST"])
def set_final_key():
    global final_key
    final_key = request.json.get("sifted_bob", None) or request.json.get("key", None)
    if final_key is None:
        return jsonify({"error": "No key provided"}), 400
    final_key = [int(b) for b in final_key]
    print("Alice: Final key received (len={}): {}".format(len(final_key), final_key))
    return jsonify({"status": "final_key_set", "length": len(final_key)})

@app.route("/get_final_key", methods=["GET"])
def get_final_key():
    global final_key
    if final_key is None:
        return jsonify({"error": "No key stored"}), 400
    return jsonify({"final_key": final_key, "length": len(final_key)})

@app.route("/send_message", methods=["POST"])
def send_message():
    global final_key
    if final_key is None:
        return jsonify({"error": "No final key available"}), 400
    plaintext = request.json.get("message", "")
    print("Alice: Encrypting and sending message:", plaintext)
    pb = plaintext.encode("utf-8")
    kb = list(final_key)
    if len(kb) == 0:
        return jsonify({"error": "Final key empty"}), 400
    # ensure at least one byte
    if len(kb) < 8:
        kb = kb + [0] * (8 - len(kb))
    key_bytes = bytes([int("".join(str(b) for b in kb[i:i+8]), 2) for i in range(0, len(kb), 8)])
    repeated_key = (key_bytes * ((len(pb) // len(key_bytes)) + 1))[:len(pb)]
    ciphertext = bytes([a ^ b for a, b in zip(pb, repeated_key)])
    try:
        r = requests.post(f"{BOB_HOST}/receive_ciphertext", json={"ciphertext": list(ciphertext)}, timeout=30)
        return jsonify({"status": "ciphertext_sent", "bob_reply": r.text})
    except Exception as e:
        print("Alice -> Bob send ciphertext failed:", e)
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    print("Alice server running on port 5000")
    app.run(host="0.0.0.0", port=5000)


Alice server running on port 5000
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.107:5000
Press CTRL+C to quit
